In [1]:
# Import những thư viện cần thiết

import numpy as np
import re
import nltk
from sklearn.datasets import load_files
import pickle
from pyvi import ViTokenizer

In [2]:
# Load training data file

training_data = load_files(r"./aug2", encoding="utf-8")
X, y = training_data.data, training_data.target

In [3]:
print(len(X))

2


In [4]:
# Load file chứa các từ stop_words

stop_ws = []
with open("vnstopword.txt", encoding="utf-8") as f :
    text = f.read()
    for word in text.split('\n') :  #Tách ra mỗi dòng là 1 từ stopword riêng lẻ
        stop_ws.append(word)
    f.close()

In [5]:
# Tiền xử lý cho tập dữ liệu training
documents = []
sum_word = 0
for sen in range(0, len(X)):
    
    # remove stop word
    document = []
    for word in str(X[sen]).split() :
        sum_word = sum_word + 1         # Đếm tổng số từ trong tập dữ liệu
        if (word not in stop_ws) :
            if ("_" in word) or (word.isalpha() == True):
                document.append(word)
    #print(document)
    document = ' '.join(document)

    # Loại bỏ Hashtag
    #document = re.sub(r'#([^\s]+)', '', str(X[sen]))
    document = re.sub(r'#([^\s]+)', '', document)
    # Loại bỏ các URL
    document = re.sub(r'http([^\s]+)', '', document)
    # Loại bỏ number có trong comment
    document = re.sub(r'\d+', '', document)
    # Loại bỏ các dấu câu, các ký tự đặc biệt (Emoji, icon cũng bị remove)
    document = re.sub(r'[^\w\s]','', document)
    # Chuyển chữ in hoa thành chữ thường
    document = document.lower()
    # Loại bỏ các ký tự dư thừa đơn lẻ (Nhiều ký tự đơn lẻ liền nhau kg xử lý được)
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    # Loại bỏ các Emoji, icon
    document = re.sub(r'[^\w\s,]', '', document)
    # Loại bỏ nhiều khoảng trắng dư thừa
     #document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Tokenization
    document = ViTokenizer.tokenize(document)
    
    document = ''.join(document)
    
    documents.append(document)
    

KeyboardInterrupt: 

In [6]:
# Số lượng câu trong tập dữ liệu
print(len(documents))

10000


In [7]:
# Số lượng word trong tập dữ liệu
print(sum_word)

1003237


In [8]:
# Converting Text to number by TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stop_ws)

X = tfidfconverter.fit_transform(documents).toarray()

C:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bất', 'gian', 'hầu', 'hết', 'kỳ', 'sự', 'thể', 'thời', 'thực', 'tất'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [9]:
# Phân chia dữ liệu thành 2 phần: Training và Testing
# X là data, y là label

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [10]:
# Xây dựng mô hình huấn luyện bằng NaiveBayes

import time
from sklearn.naive_bayes import BernoulliNB

# Thêm biến này vào để kiểm tra thời gian huấn luyện
start_NB = time.time()


classifier_NB = BernoulliNB(binarize=None)
classifier_NB.fit(X_train, y_train)

end_NB = time.time()
time_NB = end_NB-start_NB

#Show ra thời gian huấn luyện của mô hình
#print("Thời gian huấn luyện: %s second" % str(end-start))
print("Thời gian huấn luyện: ", str(time_NB), " giây")

Thời gian huấn luyện:  0.07124686241149902  giây


In [11]:
# Xây dựng mô hình huấn luyện bằng Random Forest
import time
from sklearn.ensemble import RandomForestClassifier

# Thêm biến này vào để kiểm tra thời gian huấn luyện
start_RF = time.time()

classifier_RF = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier_RF.fit(X_train, y_train) 

end_RF = time.time()
time_RF = end_RF-start_RF

#Show ra thời gian huấn luyện của mô hình
#print("Thời gian huấn luyện: %s second" % str(end-start))
print("Thời gian huấn luyện: ", str(time_RF), " giây")

Thời gian huấn luyện:  145.80549120903015  giây


In [12]:
# Xây dựng mô hình huấn luyện bằng SVM
import time

from sklearn.svm import SVC

# Thêm biến này vào để kiểm tra thời gian huấn luyện
start_SVM = time.time()

classifier_SVM = SVC(kernel = 'linear', random_state = 0)
classifier_SVM.fit(X_train, y_train)


end_SVM = time.time()
time_SVM = end_SVM-start_SVM

#Show ra thời gian huấn luyện của mô hình
#print("Thời gian huấn luyện: %s second" % str(end-start))
print("Thời gian huấn luyện: ", str(time_SVM), " giây")

Thời gian huấn luyện:  64.2354199886322  giây


In [13]:
# Predicting Sentiment

y_pred_NB = classifier_NB.predict(X_test)

y_pred_RF = classifier_RF.predict(X_test)

y_pred_SVM = classifier_SVM.predict(X_test)


In [14]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print('Accuracy of Naive Bayes:',accuracy_score(y_test, y_pred_NB))
print('Accuracy of Random Forest:',accuracy_score(y_test, y_pred_RF))
print('Accuracy of SVM:',accuracy_score(y_test, y_pred_SVM))

Accuracy of Naive Bayes: 0.8295
Accuracy of Random Forest: 0.8385
Accuracy of SVM: 0.8565


In [15]:
print('Training time of NB:',time_NB)
print('Training time of RF:',time_RF)
print('Training time of SVM:',time_SVM)

Training time of NB: 0.07124686241149902
Training time of RF: 145.80549120903015
Training time of SVM: 64.2354199886322
